In [1]:
import os
from os.path import join
import json
import openai
from IPython.display import display, Markdown

# chatOpenAI within langchain
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI

# prompt template refers to a reproducible way to generate a prompt
from langchain.prompts import ChatPromptTemplate

# LangChain for question answering over a list of documents
from langchain.chains import RetrievalQA

# Load data from a source as Document's. A Document is a piece of text and associated metadata.
from langchain.document_loaders import CSVLoader

# DocArrayInMemorySearch is a document index provided by Docarray that stores documents in memory
from langchain.vectorstores import DocArrayInMemorySearch


#takes care of storing embedded data and performing vector search for you
from langchain.indexes import VectorstoreIndexCreator

#pdf loader
from langchain.document_loaders import PyPDFLoader

# QA evaluator
from langchain.evaluation.qa import QAGenerateChain
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import CharacterTextSplitter

from langchain.chains import SequentialChain
from langchain.chains import SimpleSequentialChain
from langchain.utilities import ApifyWrapper

# environment variables

In [3]:
CURRENT_PATH = os.getcwd()
CONFIG_FILE = "config.json"

# Load Config File
with open(join(CURRENT_PATH, CONFIG_FILE)) as file:
    # Load the JSON data
    config = json.load(file)

# Environmental Variables
AZURE_OPENAI_KEY = config['AZURE_OPENAI_KEY']
AZURE_OPENAI_ENDPOINT = config['AZURE_OPENAI_ENDPOINT']
AZURE_ENGINE_NAME = config['AZURE_ENGINE_NAME']


openai_api_type = 'azure'
openai_api_key = AZURE_OPENAI_KEY

In [4]:
ai=AzureChatOpenAI(openai_api_base=AZURE_OPENAI_ENDPOINT
         ,openai_api_key=AZURE_OPENAI_KEY
         ,openai_api_type='azure'
         ,openai_api_version="2023-05-15"
         ,deployment_name =AZURE_ENGINE_NAME
         ,model = "gpt-3.5-turbo"
         ,temperature=0.1
    )

In [7]:
os.environ["APIFY_API_TOKEN"] = "apify_api_ZLchRHPERhQ0BJpgc4uvytLW31RN8U1yWqeY"
apify = ApifyWrapper()

# apify

In [8]:
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": [{"url": "https://swordhealth.com/"}]},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

In [28]:
from langchain.document_loaders.base import Document
from langchain.indexes import VectorstoreIndexCreator

os.environ["OPENAI_API_KEY"] = AZURE_OPENAI_KEY
index = VectorstoreIndexCreator().from_loaders([loader])

AuthenticationError: Incorrect API key provided: a3835264********************b3c2. You can find your API key at https://platform.openai.com/account/api-keys.

In [30]:
vectordb_chart=Chroma.from_loaders(
    texts=loader,
    embedding=embedding,
    persist_directory=persist_directory
)

AttributeError: type object 'Chroma' has no attribute 'from_loaders'

In [ ]:
query = "What is SwordHealth?"
result = index.query_with_sources(query)

# embedding & vectorstore

In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding=OpenAIEmbeddings(openai_api_key=openai_ada_key
                              , openai_api_base = openai_ada_base
                              , deployment = openai_ada_deployment
                              , openai_api_type = openai_api_type
                              ,chunk_size=1)

In [7]:
from langchain.vectorstores import Chroma

persist_directory='docs/chroma'
vectordb_chart=Chroma.from_texts(
    texts=chart_file,
    embedding=embedding,
    persist_directory=persist_directory
)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2022-12-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 7 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2022-12-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 3 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Cr

# memory

In [8]:
from langchain.memory import ConversationSummaryMemory
memory_developmentplan = ConversationSummaryMemory(memory_key="chat_history", llm=AzureChatOpenAI(openai_api_base=AZURE_OPENAI_ENDPOINT
         ,openai_api_key=AZURE_OPENAI_KEY
         ,openai_api_type='azure'
         ,openai_api_version="2023-05-15"
         ,deployment_name =AZURE_ENGINE_NAME
         ,model = "gpt-3.5-turbo"
         ,temperature=0.1
    ), return_messages=True, input_key="question")

In [8]:
from langchain.memory import ConversationSummaryMemory
memory_developmentplan = ConversationSummaryMemory(memory_key="chat_history", llm=AzureChatOpenAI(openai_api_base=AZURE_OPENAI_ENDPOINT
         ,openai_api_key=AZURE_OPENAI_KEY
         ,openai_api_type='azure'
         ,openai_api_version="2023-05-15"
         ,deployment_name =AZURE_ENGINE_NAME
         ,model = "gpt-3.5-turbo"
         ,temperature=0.1
    ), return_messages=True, input_key="question")

# option 1 - stuff

In [40]:
prompt_template = """Given the following information, give a recommendation of a development plan for someone working under the given person.
In addition, , return 5 examples of role-specific training.

Person: {question}
Context about person:{context}
Role:{role}
Context about role:{area}
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question", "area", "role"]
)
chain_nw = load_qa_chain(ai, chain_type="stuff", prompt=PROMPT, memory=memory_network, output_key="plan")

In [41]:
person="Richard Clark"
role="Business Analyst" 

In [42]:
chain_nw({"input_documents": vectordb_chart.similarity_search(person,k=3)
          ,"area": vectordb_chart.similarity_search(role,k=3)
          , "question": person, "role":role}, return_only_outputs=True)

{'people': "Development Plan Recommendation for Business Analyst working under Richard Clark:\n\n1. Strengthen Financial Analysis Skills: As a Business Analyst, it is crucial to have a strong understanding of financial analysis. Richard Clark, as the Finance Manager, can provide guidance and mentorship to enhance the Business Analyst's financial analysis skills. This can include training on financial statement analysis, budgeting, and forecasting.\n\n2. Improve Data Analysis and Interpretation: Richard Clark's expertise in financial analysis can be leveraged to enhance the Business Analyst's data analysis and interpretation skills. This can involve training on statistical analysis techniques, data visualization tools, and predictive modeling to extract meaningful insights from data.\n\n3. Enhance Strategic Decision-Making Abilities: Richard Clark's role involves strategic decision-making, and the Business Analyst can benefit from learning about the decision-making processes and framewo

In [43]:
memory_network.chat_memory.messages[-1].content

"Development Plan Recommendation for Business Analyst working under Richard Clark:\n\n1. Strengthen Financial Analysis Skills: As a Business Analyst, it is crucial to have a strong understanding of financial analysis. Richard Clark, as the Finance Manager, can provide guidance and mentorship to enhance the Business Analyst's financial analysis skills. This can include training on financial statement analysis, budgeting, and forecasting.\n\n2. Improve Data Analysis and Interpretation: Richard Clark's expertise in financial analysis can be leveraged to enhance the Business Analyst's data analysis and interpretation skills. This can involve training on statistical analysis techniques, data visualization tools, and predictive modeling to extract meaningful insights from data.\n\n3. Enhance Strategic Decision-Making Abilities: Richard Clark's role involves strategic decision-making, and the Business Analyst can benefit from learning about the decision-making processes and frameworks used by

# option 2 - map_reduce

In [35]:
question_prompt_template = """Given the following information, give a recommendation of a development plan for someone working under the given leader.

Leader: {leader}
Context about Leader:{context}
Role:{role}
Context about role:{area}
"""
QUESTION_PROMPT = PromptTemplate(
    template=question_prompt_template, input_variables=["context", "leader", "area", "role"]
)

combine_prompt_template = """Considering the following development plan, return 5 examples of role-specific training.

Development plan: {summaries}"""
COMBINE_PROMPT = PromptTemplate(
    template=combine_prompt_template, input_variables=["summaries"]
)

In [36]:
chain = load_qa_chain(ai, chain_type="map_reduce", question_prompt=QUESTION_PROMPT, combine_prompt=COMBINE_PROMPT, return_map_steps=True)

In [37]:
person="Richard Clark, Finance Manager"
role="Business Analyst" 

In [38]:
result=chain({"input_documents": vectordb_chart.similarity_search(person,k=1)
              ,"leader": person
              ,"area": vectordb_chart.similarity_search(role,k=3)
              ,"role":role}, return_only_outputs=True)

In [39]:
result

{'intermediate_steps': ["Based on the given information, a development plan for a Business Analyst working under Richard Clark, the Finance Manager, could include the following recommendations:\n\n1. Strengthen Financial Analysis Skills: As a Business Analyst, it is important to have a strong foundation in financial analysis. The individual should focus on improving their skills in areas such as financial modeling, forecasting, and budgeting. This can be achieved through attending relevant training programs or courses, seeking mentorship from experienced finance professionals, and actively participating in financial analysis projects within the organization.\n\n2. Enhance Strategic Decision-Making Abilities: Richard Clark, as the Finance Manager, is responsible for strategic decision-making. The Business Analyst should aim to develop their ability to think strategically and contribute valuable insights to the decision-making process. This can be achieved by actively engaging with Richa